# 一元配置分散分析 (One-way ANOVA analysis)
### パッケージ・モジュールをインポートする　Import packages

In [2]:
import pandas as pd
import scipy.stats as ss

import statsmodels.api as sm
from statsmodels.formula.api import ols
# dataframeを綺麗に表示する　rich display for dataframes
from IPython.core.display import display

### Rのデータセットから植物成長データ　This data is from Rdatasets

Pandasでファイルを読み込む Load data into a pandas's dataframe

In [7]:
datafile = "data/prob8.txt"
data_frame = pd.read_csv(datafile, header=0, delim_whitespace=8)
display(data_frame)

,diameter,batch
0,1.006,1.0
1,0.996,1.0
2,0.998,1.0
3,1.000,1.0
4,0.992,1.0
5,0.993,1.0
6,1.002,1.0
7,0.999,1.0
8,0.994,1.0
9,1.000,1.0


### グループの種類やデータの分割

"group"ヘッドの値を分割し、pythonの辞書型変数に代入する  split data into a python dict  
pd.unique()：ユニークのグループ名を抽出する。 Find unique group names

In [9]:
groups = pd.unique(data_frame.batch.values)
# グループに対応するデータを分割し、pythonの辞書型変数に代入する
# Split data into groups and store them into a python dict
data = {batch: data_frame['diameter'][data_frame.batch==batch] for batch in groups}

### scipyのf_oneway()関数 scipy's f_oneway() function

scipy's f_oneway():  
**定義：**  
```python  
  f_val, p_value = scipy.stats.f_oneway(*args)  

    sample1, sample2, ... : 各グループの標本データ　The sample measurements for each group.
    f_val: F値   F-statistic value
    p_value: p値 p value
```

### scipyのf_oneway()関数 scipy's f_oneway() function
使用条件：Assumption  
 1. 各標本は独立した。The samples are independent.  
 2. 各標本は正規分布に従う母集団からのものです。Each sample is from a normally distributed population.  
 3. 各グループの母集団の標準偏差はすべて等しい。いわゆる等分散性です。  
    The population standard deviations of the groups are all equal. This property is known as homoscedasticity.

In [12]:
f, p = ss.f_oneway(data[1.0], data[2.0], data[3.0], data[4.0], data[5.0], data[6.0], data[7.0], data[8.0], data[9.0], data[10.0])
print("Using f_oneway of scipy")
print("F-value: {} /p-value:{:.5f}".format(f, p))

Using f_oneway of scipy
F-value: 2.2969124133585295 /p-value:0.02266


### Using statsmodels

Step 1. statsmodelsのformulaを用いて、回帰分析を行う。  
Step 2. 回帰分析の結果に基づいて、一元配置分散分析を行う。  

<span style="color:red">aov_table</span> = **[anvova_lm](https://www.statsmodels.org/dev/generated/statsmodels.stats.anova.anova_lm.html)**(fitted_result, scale=None, test="F", typ=1, robust=None)  

**入力(input)は**  

<span style="color:blue">**fitted_result**</span>: statsmodelsのformulaの回帰結果を持つオブジェクト.  
One or more fitted linear models  

<span style="color:blue">**scale**</span>: 分散の評価です。"None"の場合、最大のモデルから評価する。  
               Estimate of variance, If None, will be estimated from the largest model.  

<span style="color:blue">**test**</span>: 統計量です。"F", "Chisq", "Cp", 或いは None.  
              Test statistics to provide.  "F", "Chisq", "Cp", or None  

<span style="color:blue">**typ**</span>: 平方和のタイプ。The type of Sum of squares to perform  
             unbalancedデータに対して, 平方和の計算方法は少なくとも３つがある。
             一般的にType I, II and IIIと呼びます。  
             For unbalanced data, there are at least 3 approaches to calculate the sums  
             of squares for ANOVA, commonly called Type I, II and III.  

<span style="color:blue">**robust**</span>: 等分散性を補正した係数の共分散を使う。  
                 Use heteroscedasticity-corrected coefficient covariance matrix.  

 **出力(output)は**  

<span style="color:red">**aov_table**</span>: pandasのDataFrameです。列は以下です。  
         "sum_sq": Sum of squares for model terms.  
         "df"    : Degrees of freedom for model terms.  
         "F"     : F statistic value for significance of adding model terms.  
         "PR(>F)": P-value for significance of adding model terms.  


In [13]:
model2 = ols(formula='diameter ~ batch', data=data_frame) # ols formulaのモデルを構築
res = model2.fit() # 回帰分析を行う
# see designed matrix
#display(res.model.endog)
#display(res.model.exog)
#display(res.model.exog_names)
print(res.summary()) # 結果を出力する

# In a numerical way
Y = res.model.endog
x = res.model.exog
#X = sm.add_constant(x)
model = sm.OLS(Y, x)
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               diameter   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                   0.04168
Date:                Thu, 22 Nov 2018   Prob (F-statistic):              0.839
Time:                        13:19:09   Log-Likelihood:                 365.69
No. Observations:                 100   AIC:                            -727.4
Df Residuals:                      98   BIC:                            -722.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9974      0.001    731.757      0.0

### 一元配置分散分析: Statsmodels' anova_lm()

In [15]:
print("----------------")
print("ANOVA table:")

# one-way anova by anova_lm()
aov_table = sm.stats.anova_lm(res, typ=2)

# aov_tableはpandasのdataframeオブジェクトです・
print("----------------")
print(aov_table)
print("----------------")
# F値の取得、 access F-statistic
print("F値：", aov_table.at['batch','F'])
# F値の取得、 access F-statistic
print("p値：", aov_table.at['batch','PR(>F)'])
# グループ間の自由度
print("グループ間の自由度:",aov_table.at['batch','df'])

----------------
ANOVA table:
----------------
            sum_sq    df         F    PR(>F)
batch     0.000002   1.0  0.041683  0.838649
Residual  0.003901  98.0       NaN       NaN
----------------
F値： 0.04168283550906771
p値： 0.8386489287247627
グループ間の自由度: 1.0


### 効果量(Effective size)

実験的研究では,効果量は独立変数（IV）が従属変数（DV）に与えた影響の程度を表す数値です。  
 Effect size is a value which allows you to see how much  
 your independent variable (IV) has affected the dependent variable (DV) in  
 an experimental study.  

 ここでは、カテゴリ独立変数に対して、より直感的な解釈であり、計算しやすい2つの尺度$\eta^2$と$\omega^2$を紹介します。  
 Here we introduce two measures, $\eta^2$ and $\omega^2$ for categorical independent variables have a more intuitive interpretation, and are easier to evaluate.  
1. 標本数が多くなると、$\eta^2$の評価はbiasが小さいとなります。  
2. 標本数が少ない場合、$\omega^2$の評価は相応しいです。  

基本的には要因による変動(各条件の平均値差)が大きいほど、 または誤差による変動(各条件の標準偏差が)小さいほど、効果量は大きくなる。  
 The bias of $\eta^2$ gets very small as sample size increases, but for small samples an unbiased effect size measure is Omega Squared.  Basically, the greater the fluctuation due to factors (difference in average value of each condition), or the smaller the fluctuation due to error (standard deviation of each condition), the larger the effect size.

### 結果　Result

In [18]:
if aov_table.at['batch','PR(>F)'] > 0.05:
    print("Retain the null hypothesis")
else:
    print("Reject the null hypothesis")

Retain the null hypothesis


### 解釈 Interpretation
5%有意水準でロットごとの母集団の平均値が全て等しいではありません。  
The population mean of each batches are not all equal at 5% level of significance.